In [1]:
import wooldridge as woo
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import acorr_breusch_godfrey as bgtest
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm
import seaborn as sns
import gretltests
import matplotlib.pyplot as plt
import numpy as np
import augmentdata

plt.rcParams['figure.figsize'] = [10, 8]


from statsmodels.tools.tools import add_constant
import linearmodels.iv.model as lm   #This had to be pip-installed first, gives diagnostic tests

from statsmodels.sandbox.regression.gmm import IV2SLS #This gives the same results as Gretl but not the diagnostic tests

# Session 16 Slides

In [2]:
data0=pd.read_csv('data13-1.csv')
data0.columns

Index(['obs', 'YEAR', 'GDP', 'CONS', 'INV', 'EXPORTS', 'IMPORTS', 'GOVEXP',
       'GOVREC', 'MONYSUP', 'Pt', 'rt', 'POP'],
      dtype='object')

In [3]:
mod1=ols('CONS~GDP',data=data0).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:                   CONS   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.546e+04
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           1.15e-45
Time:                        09:22:06   Log-Likelihood:                -170.88
No. Observations:                  35   AIC:                             345.8
Df Residuals:                      33   BIC:                             348.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -272.7589     20.977    -13.003      0.0

In [4]:
#To save on repetition lets create the following
def tsls(mod,modb):
    print('\nNote that that wooldridge_regression test is the Hausman test produced by Gretl if debiased=False\n')
    print(mod.wu_hausman())
    print('\nDebiased:',mod.wooldridge_regression)
    print('\nNot Debiased:',modb.wooldridge_regression)
    print('\n' ,mod.sargan)
    print('\nThe result from the linear models version of the regression is')
    print(mod.summary)
    return

In [5]:

Y0=data0['CONS']
X0=data0['GDP']
Z0=data0['INV']
X0=add_constant(X0)
Z0=add_constant(Z0)

spec2_=lm.IV2SLS(dependent=Y0,endog=data0['GDP'],instruments=data0['INV'],exog=Z0['const'])
mod2_=spec2_.fit(cov_type='homoskedastic',debiased=True)
mod2b=spec2_.fit(cov_type='homoskedastic',debiased=False)

tsls(mod2_,mod2b)

#Note that there is also this version but it gives the same result
#mod2=IV2SLS(Y, X, instrument=Z).fit()
#print(mod2.summary())



Note that that wooldridge_regression test is the Hausman test produced by Gretl if debiased=False

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 3.0331
P-value: 0.0912
Distributed: F(1,32)

Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 3.0331
P-value: 0.0816
Distributed: chi2(1)

Not Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 3.3175
P-value: 0.0685
Distributed: chi2(1)

 Invalid test statistic
Test requires more instruments than endogenous variables.
Sargan's test of overidentification

The result from the linear models version of the regression is
                          IV-2SLS Estimation Summary                          
Dep. Variable:                   CONS   R-squared:                      0.9979
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9978
No. Observations:                  35   F-statistic:

In [6]:
#The test for wek instruments simply the F-test from this regression (the endogneous on the instruments)
mod3=ols('GDP~INV',data=data0).fit()
print(mod3.summary())

                            OLS Regression Results                            
Dep. Variable:                    GDP   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     437.9
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           1.28e-20
Time:                        09:22:07   Log-Likelihood:                -243.18
No. Observations:                  35   AIC:                             490.4
Df Residuals:                      33   BIC:                             493.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    271.1845    158.743      1.708      0.0

In [7]:
Z1=data0[['INV','YEAR']]
Z1=add_constant(Z1)

spec4=lm.IV2SLS(dependent=Y0,endog=data0['GDP'],instruments=data0[['INV']],exog=Z1[['const','YEAR']])
mod4=spec4.fit(cov_type='homoskedastic',debiased=True)
mod4b=spec4.fit(cov_type='homoskedastic',debiased=False)


tsls(mod4,mod4b)


Note that that wooldridge_regression test is the Hausman test produced by Gretl if debiased=False

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.9024
P-value: 0.3495
Distributed: F(1,31)

Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 0.9024
P-value: 0.3422
Distributed: chi2(1)

Not Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 1.0188
P-value: 0.3128
Distributed: chi2(1)

 Invalid test statistic
Test requires more instruments than endogenous variables.
Sargan's test of overidentification

The result from the linear models version of the regression is
                          IV-2SLS Estimation Summary                          
Dep. Variable:                   CONS   R-squared:                      0.9983
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9982
No. Observations:                  35   F-statistic:

In [8]:
spec5=lm.IV2SLS(dependent=Y0,endog=data0['GDP'],instruments=data0[['INV','YEAR']],exog=Z1[['const']])
mod5=spec5.fit(cov_type='homoskedastic',debiased=True)
mod5b=spec5.fit(cov_type='homoskedastic',debiased=False)


tsls(mod5,mod5b)


Note that that wooldridge_regression test is the Hausman test produced by Gretl if debiased=False

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 11.0867
P-value: 0.0022
Distributed: F(1,32)

Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 3.8222
P-value: 0.0506
Distributed: chi2(1)

Not Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 4.1805
P-value: 0.0409
Distributed: chi2(1)

 Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 5.6483
P-value: 0.0175
Distributed: chi2(1)

The result from the linear models version of the regression is
                          IV-2SLS Estimation Summary                          
Dep. Variable:                   CONS   R-squared:                      0.9979
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9978
No. Observations:                  35   

In [9]:
#The test for weak instruments simply the F-test from this regression (the endogeneous on the instruments)
#Note that in general the exogneous variables need to be included  but here there is only the constant
mod6=ols('GDP~INV+YEAR',data=data0).fit()
print(mod6.summary())


                            OLS Regression Results                            
Dep. Variable:                    GDP   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     2705.
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           2.04e-36
Time:                        09:22:07   Log-Likelihood:                -199.82
No. Observations:                  35   AIC:                             405.6
Df Residuals:                      32   BIC:                             410.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.519e+05   8143.431    -18.658      0.0

# Session 16 Interactive

In [10]:
data=pd.read_csv('truffles.csv')

In [11]:
mod7=ols('q~p+ps+di',data=data).fit()
print(mod7.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.438
Method:                 Least Squares   F-statistic:                     8.520
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           0.000416
Time:                        09:22:08   Log-Likelihood:                -77.657
No. Observations:                  30   AIC:                             163.3
Df Residuals:                      26   BIC:                             168.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0910      3.712      0.294      0.7

In [12]:
Z=data[['ps','di']]
Z=add_constant(Z)
spec7=lm.IV2SLS(dependent=data['q'],endog=data['p'],instruments=data['pf'],exog=Z[['const','ps','di']])
mod7=spec7.fit(cov_type='homoskedastic',debiased=True)
mod7b=spec7.fit(cov_type='homoskedastic',debiased=False)


tsls(mod7,mod7b)


Note that that wooldridge_regression test is the Hausman test produced by Gretl if debiased=False

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 110.4034
P-value: 0.0000
Distributed: F(1,25)

Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 110.4034
P-value: 0.0000
Distributed: chi2(1)

Not Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 132.4841
P-value: 0.0000
Distributed: chi2(1)

 Invalid test statistic
Test requires more instruments than endogenous variables.
Sargan's test of overidentification

The result from the linear models version of the regression is
                          IV-2SLS Estimation Summary                          
Dep. Variable:                      q   R-squared:                     -0.0239
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1421
No. Observations:                  30   F-stat

In [13]:
#The weak instrument test is simply the regression of the endogenous variable below with the F-test
#The F is simply the t value squared
mod8=ols('p~ps+di+pf',data=data).fit()
print('Weak Instrument Test',[(mod8.tvalues**2)['pf']])
#print(mod8.summary())


Weak Instrument Test [20.571696314228614]


In [14]:
data['p_lag']=data['p'].shift(1)
Z=data[['ps','di']]
Z=add_constant(Z)
spec9=lm.IV2SLS(dependent=data['q'],endog=data['p'],instruments=data[['pf','p_lag']],exog=Z[['const','ps','di']])
mod9=spec9.fit(cov_type='homoskedastic',debiased=True)
mod9b=spec9.fit(cov_type='homoskedastic',debiased=False)



tsls(mod9,mod9b)


Note that that wooldridge_regression test is the Hausman test produced by Gretl if debiased=False

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 96.3229
P-value: 0.0000
Distributed: F(1,24)

Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 94.3343
P-value: 0.0000
Distributed: chi2(1)

Not Debiased: Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 113.9873
P-value: 0.0000
Distributed: chi2(1)

 Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 0.1139
P-value: 0.7358
Distributed: chi2(1)

The result from the linear models version of the regression is
                          IV-2SLS Estimation Summary                          
Dep. Variable:                      q   R-squared:                     -0.0612
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1886
No. Observations:                  29

C:\Users\aes05kgb\Anaconda3\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [15]:
#The weak instrument test is:
#The weak instrument test is simply the regression of the endogenous variable below with the F-test
#The F is simply the t value squared
mod8=ols('p~ps+di+pf+p_lag',data=data).fit()
restrictions='(pf=0),(p_lag=0)'
mod8.f_test(restrictions)
#print(mod8.summary())

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[7.8410916]]), p=0.002395456052260628, df_denom=24, df_num=2>

In [16]:
#EXTRA
#You can also replicate the Gretl Results as follows, but not the diagnostics. 
#Note the difference in the way it specifies the regs
data=add_constant(data)
mod9=IV2SLS(data['q'], data[['const','di','ps','p']], instrument=data[['const','di','ps','pf']]).fit()
print(mod9.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      q   R-squared:                      -0.024
Model:                         IV2SLS   Adj. R-squared:                 -0.142
Method:                     Two Stage   F-statistic:                     5.903
                        Least Squares   Prob (F-statistic):            0.00327
Date:                Sat, 07 Jan 2023                                         
Time:                        09:22:08                                         
No. Observations:                  30                                         
Df Residuals:                      26                                         
Df Model:                           3                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2795      5.544     -0.772      0.4

In [17]:
data_=data.iloc[1:,:]
mod10=IV2SLS(data_['q'], data_[['const','di','ps','p']], instrument=data_[['const','di','ps','pf','p_lag']]).fit()
print(mod10.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      q   R-squared:                      -0.061
Model:                         IV2SLS   Adj. R-squared:                 -0.189
Method:                     Two Stage   F-statistic:                     5.449
                        Least Squares   Prob (F-statistic):            0.00505
Date:                Sat, 07 Jan 2023                                         
Time:                        09:22:08                                         
No. Observations:                  29                                         
Df Residuals:                      25                                         
Df Model:                           3                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2269      5.743     -0.736      0.4